<a href="https://colab.research.google.com/github/oAlanSchmitt/CatalogAI/blob/main/CatalogAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALAÇÃO DO SDK

In [1]:
!pip install -q -U google-generativeai

IMPORTANDO BIBLIOTECAS

In [2]:
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai

CONFIGURAÇÃO DA API_KEY

In [4]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

CONFIGURAÇÕS DO MODELO

In [5]:
# --- Configuração do modelo ---
generation_config = {"candidate_count": 1, "temperature": 0.7}

# É altamente recomendável manter os filtros de segurança ativados!
safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

# Inicializando o modelo
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config=generation_config,
    safety_settings=safety_settings,
)

# Instrução do sistema, modificada para "Chances de Você Gostar"
system_instruction = """
Você é um assistente especializado em recomendar animes, filmes e séries.

Siga estas regras:

* **Recomendações únicas:** Nunca sugira o mesmo título que o usuário forneceu como entrada.
* **Diversidade:** Recomende um filme, um anime e uma série.
* **Formato da resposta:**
    * Filme - Título: [Título do filme]
    * Filme - Sinopse: [Breve sinopse do filme]
    * Filme - Chances de Você Gostar: [Alta, Média ou Baixa]
    * Anime - Título: [Título do anime]
    * Anime - Sinopse: [Breve sinopse do anime]
    * Anime - Chances de Você Gostar: [Alta, Média ou Baixa]
    * Série - Título: [Título da série]
    * Série - Sinopse: [Breve sinopse da série]
    * Série - Chances de Você Gostar: [Alta, Média ou Baixa]

Exemplo:

Informação: The Walking Dead, Guerra Mundial Z, Zumbilândia, John Wick, Mad Max.
Resposta:
    Filme - Título: Army of the Dead
    Filme - Sinopse: Um grupo de mercenários planeja um assalto a um cassino em Las Vegas durante um surto de zumbis.
    Filme - Chances de Você Gostar: Alta
    Anime - Título: Highschool of the Dead
    Anime - Sinopse: Um grupo de estudantes precisa lutar para sobreviver a um apocalipse zumbi.
    Anime - Chances de Você Gostar: Média
    Série - Título: Kingdom
    Série - Sinopse: Um príncipe coreano precisa enfrentar uma misteriosa praga zumbi que está devastando o reino.
    Série - Chances de Você Gostar: Alta
"""

BASE DO MODELO

In [6]:
chat = model.start_chat(history=[])

# --- Interação com o usuário ---
display(Markdown("##  ✨ Bem-vindo ao CatalogAI! ✨"))
display(Markdown("Sou seu assistente para descobrir novos filmes 🎬, animes 🏯 e séries 📺! "))
display(Markdown("Vou usar meu conhecimento para te recomendar títulos que combinem com o seu gosto! 🧠💡"))
display(Markdown(""))
display(Markdown("Conte-me sobre 5 títulos que você curte 💖 (pode ser qualquer tipo! 😄):"))
display(Markdown(""))
display(Markdown("### Por exemplo: "))
display(Markdown("- Matrix"))
display(Markdown("- Attack on Titan"))
display(Markdown("- Stranger Things"))

# Coleta de 5 títulos
titulos = []
for i in range(5):
    titulo = input(f"Título {i+1}: ")
    titulos.append(titulo)

ip1, ip2, ip3, ip4, ip5 = titulos

# --- Loop de recomendações ---
while True:
    prompt = f"Informação: {', '.join(titulos)}.\nResposta:\n"
    response = chat.send_message(system_instruction + prompt)

    # --- Processamento da resposta ---
    try:
        resposta = {}
        linhas = response.text.split('\n')
        for linha in linhas:
            if ':' in linha:
                chave, valor = linha.split(':', 1)
                resposta[chave.strip()] = valor.strip()

        # --- Exibindo a resposta formatada dentro do loop ---
        for tipo in ["Filme", "Anime", "Série"]:
            titulo = resposta.get(f"{tipo} - Título", 'Título não encontrado')
            sinopse = resposta.get(f"{tipo} - Sinopse", 'Sinopse não disponível')
            chances = resposta.get(f"{tipo} - Chances de Você Gostar", 'Chances não disponíveis')

            display(Markdown(f""))
            display(Markdown(f"## **{tipo}: {titulo}**"))
            display(Markdown(f"_{sinopse}_"))
            display(Markdown(f"**Chances de Você Gostar:** {chances}"))
            display(Markdown(f""))
    except Exception as e:
        print(f"Erro ao processar a resposta: {e}")
        resposta = None

    # --- Perguntar ao usuário se deseja outra recomendação ---
    outra_recomendacao = input("Deseja outra recomendação? (s/n): ").lower()
    if outra_recomendacao != 's':
        break

##  ✨ Bem-vindo ao CatalogAI! ✨

Sou seu assistente para descobrir novos filmes 🎬, animes 🏯 e séries 📺! 

Vou usar meu conhecimento para te recomendar títulos que combinem com o seu gosto! 🧠💡

Conte-me sobre 5 títulos que você curte 💖 (pode ser qualquer tipo! 😄):

### Por exemplo: 

- Matrix

- Attack on Titan

- Stranger Things

KeyboardInterrupt: Interrupted by user